In [1]:
from pandas import DataFrame, read_csv
import matplotlib.pyplot as plt
import pandas as pd

In [30]:
def getExamplesDf(path):
    df = pd.read_csv(filepath_or_buffer=path, header=None)
    return pd.DataFrame({
        'id': df.index,
        'class': df[22],
    })
examplesDf = getExamplesDf('../../datasets/test.csv')
examplesSize = examplesDf.shape[0]
# examplesDf

In [3]:
def getMatchesDf(path):
    df = pd.read_csv(filepath_or_buffer=path)
    return pd.DataFrame({
        'id': df['#pointId'].astype(int),
        'label': df['label'],
    })
matchesDf = getMatchesDf('../../out/serial.csv')
# matchesDf

,id,label
0,0,N
1,1,N
2,2,N
3,3,N
4,4,N
...,...,...
653452,653452,N
653453,653453,N
653454,653454,N
653455,653455,N


In [31]:
def getOriginalMatchesDf(path):
    df = pd.read_table(filepath_or_buffer=path, header=None)
    df.columns=['id', 'class', 'label']
    df = df[df['id'].str.startswith('Ex:')]
    def cleanLabel(text):
        label = text.replace('Classe MINAS:', '').strip()
        if label == 'Unk': return '-'
        if label.startswith('C '): return label.replace('C ', '')
        return label
    # ogdf['class'] = cleanStrCol(ogdf['class'], 'Classe Real:')
    # ogdf['label'] = cleanStrCol(ogdf['label'], 'Classe MINAS:')
    return pd.DataFrame({
        'id': df['id'].apply(lambda x: x.replace('Ex:', '').strip()).astype(int) - 1,
        'label': df['label'].apply(cleanLabel),
        # 'class': df['class'].apply(lambda x: x.replace('Classe Real:', '').strip()),
    })
# ogdf = getOriginalMatchesDf('../../out/minas-og/2020-07-17T18-24-02.629/results')
# print(ogdf['label'].unique())
# ogdf

['N' '-']


In [32]:
# /home/puhl/project/minas-flink/mfog-mpi/out/minas-og/2020-07-20T12-21-54.755/matches.csv
# ogndDf = getOriginalMatchesDf('../../out/minas-og/2020-07-17T17-41-09.209/results')
# ogndDf['label'].unique()

ogKmSansND = getMatchesDf('../../out/minas-og/2020-07-20T12-21-54.755/matches.csv')
print(ogKmSansND['label'].unique())

# ogKmND = getMatchesDf('../../out/minas-og/2020-07-20T12-18-21.758/matches.csv')
# print(ogKmND['label'].unique())

# ogKmND_nonMatch = getOriginalMatchesDf('../../out/minas-og/2020-07-20T12-18-21.758/results')
# print(ogKmND_nonMatch['label'].unique())

['N' '-']


In [7]:
def merge(exDf, maDf):
    assert exDf.columns.all(['id', 'class'])
    assert maDf.columns.all(['id', 'label'])
    return pd.merge(exDf, maDf, on='id', how='left')[['id', 'class', 'label']]
def confusionMatrix(merged):
    assert merged.columns.all(['id', 'class', 'label'])
    return pd.crosstab(merged['class'], merged['label'],
                       rownames=['Classes (act)'], colnames=['Labels (pred)']).transpose()

In [8]:
def printEval(exDf, maDf):
    df = merge(exDf, maDf)
    confusion_matrix = confusionMatrix(df)
    # confusion_matrixP = pd.DataFrame(confusion_matrix)
    # confusion_matrixP['total'] = sum([confusion_matrixP[col] for col in confusion_matrixP.columns])
    # confusion_matrixP = confusion_matrixP.transpose()
    # confusion_matrixP['total'] = sum([confusion_matrixP[col] for col in confusion_matrixP.columns])
    print("Confusion Matrix")
    print(confusion_matrix)
    
    totalExamples = exDf['id'].count()
    totalMatches = maDf.shape[0]
    tot = max(totalMatches, totalExamples)
    hits = df[df['label'] == df["class"]]['id'].count()
    misses = df[df['label'] != df["class"]]['id'].count()

    print('Total examples   %8d' % (totalExamples))
    print('Total matches    %8d' % (totalMatches))
    print('Hits             %8d (%10f%%)' % (hits, (hits/tot) * 100.0))
    print('Misses           %8d (%10f%%)' % (misses, (misses/tot) * 100.0))
    print('Hits + Misses    %8d (%10f%%)' % (hits + misses, ((hits + misses)/tot) * 100.0))
    print('')
    return confusion_matrix

In [34]:
# printEval(examplesDf, matchesDf)
# printEval(examplesDf, ogdf)
# printEval(examplesDf, ogndDf)
minasDf = ogKmSansND
minasMatches = minasDf.shape[0]
minasMatches

653457

In [36]:
cf = confusionMatrix(merge(examplesDf, ogKmND_nonMatch))
cfClasses = cf.columns
cf['assinged class'] = cf[['A', 'N']].idxmax(axis=1)
cf['max'] = cf[cf.columns].max(axis=1)
cf

Classes (act),A,N,assinged class,max
Labels (pred),,,,
-,3774,8206,N,8206
ExtCon N,183903,98869,A,183903
ExtNov 1,123,0,A,123
ExtNov 10,2349,4066,N,4066
ExtNov 11,71,23,A,71
ExtNov 12,25,0,A,25
ExtNov 2,145,79,A,145
ExtNov 3,368,44,A,368
ExtNov 4,8,0,A,8


In [41]:
cf.columns.unique()

Index(['A', 'N', 'assinged class', 'max'], dtype='object', name='Classes (act)')

In [13]:
confusionMatrix(merge(examplesDf, ogKmND))

Classes (act),A,N
Labels (pred),,
-,3774,8206
1,123,0
10,3520,5130
11,71,289
12,26,0
2,152,82
3,368,44
4,8,0
5,82,1


In [14]:
confusionMatrix(merge(examplesDf, ogKmSansND))

Classes (act),A,N
Labels (pred),,
-,102688,94458
N,344491,111820


In [59]:
print("Original")
# og = getMatchesDf('../../out/minas-og/2020-07-20T12-21-54.755/matches.csv')
matchesDf = getMatchesDf('../../out/serial.csv')
og = getOriginalMatchesDf('../../out/minas-og/2020-07-20T18-36-59.803/results')
printEval(examplesDf, og)
print("\n\nMPI")
printEval(examplesDf, matchesDf)

Original
Confusion Matrix
Classes (act)       A       N
Labels (pred)                
-               19677     340
N              427502  205938
Total examples     653457
Total matches      653457
Hits               205938 ( 31.515157%)
Misses             447519 ( 68.484843%)
Hits + Misses      653457 (100.000000%)



MPI
Confusion Matrix
Classes (act)       A       N
Labels (pred)                
-               19677     340
N              427502  205938
Total examples     653457
Total matches      653457
Hits               205938 ( 31.515157%)
Misses             447519 ( 68.484843%)
Hits + Misses      653457 (100.000000%)



Classes (act),A,N
Labels (pred),,
-,19677,340
N,427502,205938


In [60]:
og['og'] = og['label']
m = pd.merge(og, matchesDf, on='id', how='left')#[['id', 'og', 'label']]
m[m['label_x'] != m['label_y']]

,id,label_x,og,label_y
